# 2. 載入預訓練模型與資料集

在本筆記本中，我們將學習如何從 Hugging Face Hub 載入一個預訓練的大型語言模型 (LLM) 及其對應的 Tokenizer。同時，我們也會載入一個用於後續微調任務的資料集。

## 步驟 1: 匯入必要的函式庫

首先，我們從 `transformers` 和 `torch` 匯入本次操作需要的模組。

- `AutoTokenizer`: 一個方便的類別，可以自動根據模型名稱實例化對應的 Tokenizer。
- `AutoModelForCausalLM`: 一個用於載入自回歸語言模型（如 Llama-2）的類別。
- `BitsAndBytesConfig`: 用於設定模型量化，這是實現 QLoRa 的關鍵。
- `torch`: PyTorch 函式庫。


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig


## 步驟 2: 設定模型與 Tokenizer

我們將使用 Meta 開源的 `Llama-2-7b-chat-hf` 模型作為我們的基礎模型。

為了在記憶體有限的環境（如單張消費級 GPU）中載入這個 70 億參數的模型，我們將採用 **4-bit 量化**技術。`BitsAndBytesConfig` 讓我們可以精確地設定量化細節：

- `load_in_4bit=True`: 啟用 4-bit 量化。
- `bnb_4bit_quant_type="nf4"`: 使用 NF4 (NormalFloat 4) 量化類型，這是一種專為常態分佈權重設計的先進量化方法。
- `bnb_4bit_compute_dtype=torch.bfloat16`: 在計算過程中（如矩陣乘法），使用 bfloat16 類型以保持精度和穩定性。

**注意**: 第一次下載模型權重時，會需要一些時間。


In [ ]:
model_id = "meta-llama/Llama-2-7b-chat-hf"

# 設定量化參數
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# 載入 Tokenizer
# 注意：你需要先到 Hugging Face 網站申請 Llama-2 的使用權限，並使用 `huggingface-cli login` 登入
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token # 設定 pad_token

# 載入量化後的模型
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto", # 自動將模型分配到可用硬體
)

print("模型與 Tokenizer 載入成功！")
print(model)


## 步驟 3: 載入資料集

現在模型已經準備好，我們需要一個資料集來進行微調。這裡我們使用 `mlabonne/guanaco-llama2-1k`，這是一個已經格式化好的、適合 Llama-2 指令微調的小型資料集。

我們使用 `datasets` 函式庫中的 `load_dataset` 來載入它。


In [ ]:
from datasets import load_dataset

dataset_name = "mlabonne/guanaco-llama2-1k"
dataset = load_dataset(dataset_name, split="train")

print("資料集載入成功！")
print(dataset)
print("
範例資料：")
print(dataset[0])



模型和資料都已備妥！在下一個筆記本 `03-apply-lora.ipynb` 中，我們將正式開始進行 LoRA 微調。
